In [222]:
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [223]:
df = pd.read_csv('spotify-2023.csv',encoding='latin-1', sep=',')
df.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


##подготовка данных и построение графиков

1. Напишем фнкцию change_auditions
    - внеобходима для создания словаря {год: количество прослушиваний}
    - вызывается без аргументов
2. Запустим функцию change_auditions по ее результатам построим график
    - ось X - года (последние 20)
    - ось Y - количество прослушиваний



In [224]:
def change_auditions():
    '''  создание словаря по году и сумме прослушиваний в этот год
    :params: None
    :return: res: dict
    :example: res: {2022: 489384343443, 2023: 4389894585455}
    '''

    last_10_years = sorted(df["released_year"].unique())[-20:]
    res = {}
    for i in last_10_years:
        res[i] = df.loc[df['released_year'] == i]['streams'].astype(int).sum()
    return res

In [225]:
res = change_auditions()
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(res.keys()), y=list(res.values())))
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="динамика прослушиваний песен",
                  xaxis_title="последние 20 лет",
                  yaxis_title="прослушивания за год",
                  margin=dict(l=0, r=0, t=30, b=0))

fig.show()

- [x] подготовили данные
- [x] построили таблицу

1. Отсортируем артистов по просушиваниям
    - создадим dataframe из артиста и прослушивания
    - созздадим и заполним словарь {артист: сумма прослушиваний} для этого создадим функцию top_artists
    - отсортируем словарь по убыванию значений
2. Создадим карту рассеяния
    - ось X - исполнитель / артист
    - прослушивания

In [226]:
artist_streems = df[["artist(s)_name", "streams"]]
artist_streems.head()

,artist(s)_name,streams
0,"Latto, Jung Kook",141381703
1,Myke Towers,133716286
2,Olivia Rodrigo,140003974
3,Taylor Swift,800840817
4,Bad Bunny,303236322


In [227]:
def top_artists():
    res = {}
    for index, artist, streems in artist_streems.itertuples():

        if streems.isdigit():
            if artist in res:
                res[artist] += int(streems)
            else:
                res[artist] = int(streems)
    return dict(sorted(res.items(), key=lambda i: i[1], reverse=True))

xxx = top_artists()

In [228]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(xxx.keys()),
                    y=list(xxx.values()),
                    marker=dict(opacity=0.9,
                                reversescale=True,
                                colorscale='Blues',
                                size=5),
                    line=dict(width=0.02),
                    mode='markers',


                          ),

             )
fig.update_layout(title="прослушивание треков у артиста",
                  xaxis_title="артисты",
                  yaxis_title="прослушивания", )
fig.show()

- [x] отсортировать артистов
- [x] построить график
- [ ] написать выводы

### краткий вывод
***были подготовлены данные, по которым, были пострены два грфика, из первого можно увидеть резкий скачек прослушиваний, у этого может быть масса причин, однако, можно предположить, что это связано с самоизоляцией. В 2023 году мы видем спад прослушиваний, однако год еще не законцен и данные могут значительно изменится.***

## поиск признаков с уникальными значениями

1. найдем количество уникальных элементов в столбце с помощью функции nunique
2. переведем результат в словарь {признак: количество уникальных значений}
3. отсортируем словарь по значению по убыванию
4. возьмем первые два элементв ключей

In [229]:
most_pop = df.nunique().to_dict()
most_pop = sorted(most_pop.items(), key=lambda i: i[1], reverse=True)
most_pop[:2] # найдем самые поулярные признаки

[('streams', 949), ('track_name', 943)]

## разбитие на категории / класстеры


***
Разобьем на кластеры столбец "streams". Сделаем 10 кластеров по количеству прослушиваний (10 кластер - больше всего прослушиваний 1 - меньше всего).
Разобьем на кластеры столбец "in_spotify_playlists". Сделаем 10 кластеров по количеству плейлистов (10 кластер - больше всего плейлистов 1 - меньше всего).

Были выбраны эти столбцы для проверки гипотезы:
количество стримов и количество плейлистов сильно коррелируют.
***

In [230]:
df = df.loc[df['streams'].str.isdigit()] # убрали мусор
df['streams'] = df['streams'].astype (int)

<ipython-input-230-263da5d9d277>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [231]:
bin_labels_5 = [i + 1 for i in range(10)]

df['index_stream'] = pd.qcut(df['streams'],
                              q=[i / 10 for i in range(11)],
                              labels=bin_labels_5)

In [232]:
bin_labels_5 = [i + 1 for i in range(10)]

df['playlist_sp_index'] = pd.qcut(df['in_spotify_playlists'],
                              q=[i / 10 for i in range(11)],
                              labels=bin_labels_5)

In [233]:
df

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,index_stream,playlist_sp_index
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,Major,80,89,83,31,0,8,4,3,2
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,Major,71,61,74,7,0,10,4,3,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,Major,51,32,53,17,0,31,6,3,4
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,Major,55,58,72,11,0,11,15,8,9
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,Minor,65,23,80,14,63,11,6,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,Major,60,24,39,57,0,8,3,2,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,Major,42,7,24,83,1,12,6,3,4
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,Major,80,81,67,4,0,8,6,2,2
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,Major,82,67,77,8,0,12,5,3,4


####***создали два новых признака***

#построение графиков с добавлением новых признаков

1. Создать функцию для вычисления среднего значения индекса добавления в плейлисты по годам
3. Добавить размерность в график, который был построен ранее

In [235]:
def mean_index_playlist():
  df['index_stream'] = df['index_stream'].astype (int)
  df['playlist_sp_index'] = df['playlist_sp_index'].astype (int)
  result = []
  for i in list(res.keys()):
    result.append(round(df.loc[df['released_year'] == i, 'playlist_sp_index'].mean()))
  return result


mean_index_playlist()

[10, 10, 8, 6, 9, 10, 8, 9, 8, 9, 9, 9, 8, 8, 8, 8, 7, 7, 5, 3]

In [236]:
fig = px.line_3d(df, x=list(res.keys()), y=list(res.values()), z=mean_index_playlist(), title='прослушивание и плейлисты')

fig.show()

In [237]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=list(xxx.keys()),
                    y=list(xxx.values()),
                    z=df['playlist_sp_index'],
                    marker=dict(opacity=0.9,
                                reversescale=True,
                                colorscale='Blues',
                                size=5),
                    line=dict(width=0.02),
                    mode='markers',

                          ),

             )


fig.update_layout(title="прослушивание треков у артиста и добавление в плейлисты",

                  )


fig.show()

In [238]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=list(xxx.keys()),
                    y=df['index_stream'],
                    z=df['playlist_sp_index'],
                    marker=dict(opacity=0.9,
                                reversescale=True,
                                colorscale='Blues',
                                size=5),
                    line=dict(width=0.02),
                    mode='markers',

                          ),

             )


fig.update_layout(title="артист индекс прослушиваний и индекс плейлистов")

fig.show()

# создание высотной карты (дополнительное задание)
1. создадим высотную карту
    - ось X - индекс прослушиваний
    - ось Y - индекс плейлистов
    - ось Z - количество песен у которых индекс прослушиваний == X индекс плейлистов == Y


In [240]:
matrix = []
for i in range(1, 11):
  line = []
  for j in range(1, 11):
    line.append(len(df[(df['index_stream']==i) &(df['playlist_sp_index']==j)]))
  matrix.append(line)

fig = go.Figure(go.Surface(
    contours = {
        "x": {"show": True, "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
        "z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.05}
    },
    x = [i for i in range(1, 11)],
    y = [i for i in range(1, 11)],
    z = matrix))
fig.update_layout(title="высотная карта индексов прослушивания и добавления в плейлист" )

fig.show()

### рассчет корреляции и выводы

In [242]:
df['index_stream'].corr(df['playlist_sp_index']) # корреляция

0.827113480578827

##***корреляция 0.8 - сильная -> гипотеза подтверждена***